![](https://i.stack.imgur.com/FdqXQ.png)

In [24]:
import math
import cairo

WIDTH, HEIGHT = 512, 2048

surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
ctx = cairo.Context(surface)

ctx.set_source_rgb(0, 0, 0)
ctx.paint()

ctx.select_font_face('CMUSansSerif', cairo.FONT_SLANT_NORMAL, cairo.FONT_WEIGHT_BOLD)
ctx.set_font_size(100)

chars = ''.join(
    [chr(i) for i in range(ord('a'), ord('z') + 1)] +
    [chr(i) for i in range(ord('а'), ord('я') + 1)] + ['ё']
)
chars += chars.upper()
chars += '0123456789:;.,?!@#$%&^*+~-=_[](){}<>\\|/"\'`«»—–… '

assert(len(chars) == len(set(chars)))

stroke_width = 10

chars_info = {
    'width': WIDTH,
    'height': HEIGHT,
    'image': 'cmu_serif.png',
    'name': 'cmu_serif',
}

max_above = 0
max_below = 0

by_height = []

for char in chars:
    extents = ctx.text_extents(char)
    char_info = {}
    chars_info[char] = char_info
    char_info['x_bearing'] = extents.x_bearing
    char_info['y_bearing'] = extents.y_bearing
    char_info['width'] = extents.width
    char_info['height'] = extents.height
    char_info['x_advance'] = extents.x_advance
    char_info['y_advance'] = extents.y_advance
    by_height.append((char_info['height'], char))

by_height.sort()
    
pad = stroke_width * 0.580

lines = ['']
cur_line_width = 0

for _, char in by_height:
    char_info = chars_info[char]
    width = char_info['width'] + pad * 2
    if cur_line_width + width > WIDTH:
        lines.append('')
        cur_line_width = 0
    cur_line_width += width
    lines[-1] += char
    
ctx.set_operator(cairo.Operator.ADD)

cur_x = pad
cur_y = pad

for line in lines:
    max_height = 0
    for char in line:
        char_info = chars_info[char]
        char_info['tex_x'] = cur_x - pad
        char_info['tex_y'] = cur_y - pad
        char_info['tex_w'] = char_info['width'] + pad * 2
        char_info['tex_h'] = char_info['height'] + pad * 2
        char_info['pad'] = pad
        ctx.move_to(cur_x - char_info['x_bearing'], cur_y - char_info['y_bearing'])
        max_height = max(max_height, char_info['height'])
        cur_x += char_info['width'] + pad * 2
        ctx.text_path(char)
        ctx.set_source_rgb(1, 0, 0)
        ctx.set_line_width(10)
        ctx.set_line_join(cairo.LineJoin.ROUND)
        ctx.stroke_preserve()
        ctx.fill_preserve()
        ctx.set_source_rgb(0, 1, 0)
        ctx.fill()
    cur_x = pad
    cur_y += max_height + pad * 2

surface.write_to_png('cmu_serif.png')
surface.write_to_png('../renderer/cmu_serif.png')

import json
json.dump(chars_info, open('cmu_serif.json', 'w'))